# <center>Идентификация пользователей по посещенным веб-страницам

В этом проекте мы будем решать задачу идентификации пользователя по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в [статье](https://habrahabr.ru/company/yandex/blog/230583/) на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".


Мы будем решать похожую задачу: по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком, мы будем идентифицировать этого человека. Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

Будем использовать данные из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data". И хотя мы не можем рекомендовать эту статью (описанные методы делеки от state-of-the-art, лучше обращаться к [книге](http://www.charuaggarwal.net/freqbook.pdf) "Frequent Pattern Mining" и последним статьям с ICDM), но данные там собраны аккуратно и представляют интерес.

Имеются данные с прокси-серверов Университета Блеза Паскаля, они имеют очень простой вид. Для каждого пользователя заведен csv-файл с названием user\*\*\*\*.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: <br>

<center>*timestamp, посещенный веб-сайт*</center>

Скачать исходные данные можно по ссылке в статье, там же описание.
Для этого задания хватит данных не по всем 3000 пользователям, а по 10 и 150. [Ссылка](https://drive.google.com/file/d/1AU3M_mFPofbfhFQa_Bktozq_vFREkWJA/view?usp=sharing) на архив *capstone_user_identification* (~7 Mb, в развернутом виде ~ 60 Mb). 

В финальном проекте уже придется столкнуться с тем, что не все операции можно выполнить за разумное время (скажем, перебрать с кросс-валидацией 100 комбинаций параметров случайного леса на этих данных Вы вряд ли сможете), поэтому мы будем использовать параллельно 2 выборки: по 10 пользователям и по 150. Для 10 пользователей будем писать и отлаживать код, для 150 – будет рабочая версия. 

Данные устроены следующем образом:

 - В каталоге 10users лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя;
 - Аналогично для каталога 150users – там 150 файлов;
 - В каталоге 3users – игрушечный пример из 3 файлов, это для отладки кода предобработки, который Вы далее напишете.

# <center>Часть 1. Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы `Scipy.sparse`), который хорошо подходит для данной задачи. 

**План 1 части:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

**В задании будут использоваться библиотеки Python [`glob`](https://docs.python.org/3/library/glob.html), [`pickle`](https://docs.python.org/2/library/pickle.html) и класс [`csr_matrix`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html) из `Scipy.sparse`.**

Наконец, для лучшей воспроизводимости результатов приведем список версий основных используемых в проекте библиотек: NumPy, SciPy, Pandas, Matplotlib, Statsmodels и Scikit-learn. Для этого воспользуемся расширением [watermark](https://github.com/rasbt/watermark). Рекомендуется использовать докер-контейнер открытого курса OpenDataScience по машинному обучению, инструкции [тут](https://goo.gl/RrwpNd).

In [1]:
# pip install watermark
%load_ext watermark

In [2]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 2.7.14
IPython 5.4.1

numpy 1.14.0
scipy 1.0.0
pandas 0.22.0
matplotlib 2.1.2
statsmodels 0.8.0
sklearn 0.19.1

compiler   : GCC 7.2.0
system     : Linux
release    : 4.15.0-23-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : dae3d9d6788ca600454a924c455333440d25db4e


In [3]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [6]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = '../data'

In [7]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '10users/user0031.csv'))

In [8]:
user31_data.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами *path_to_csv_files* и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

Детали:
- Смотрите чуть ниже пример вывода, что должна возвращать функция
- Используйте `glob` (или аналоги) для обхода файлов в каталоге. Для определенности, отсортируйте список файлов лексикографически. Удобно использовать `tqdm_notebook` (или просто `tqdm` в случае python-скрипта) для отслеживания числа выполненных итераций цикла
- Создайте частотный словарь уникальных сайтов (вида {'site_string': (site_id, site_freq)}) и заполняйте его по ходу чтения файлов. Начните с 1
- Рекомендуется меньшие индексы давать более часто попадающимся сайтам (приницип наименьшего описания)
- Не делайте entity recognition, считайте *google.com*, *http://www.google.com* и *www.google.com* разными сайтами (подключить entity recognition можно уже в рамках индивидуальной работы над проектом)
- Скорее всего в файле число записей не кратно числу *session_length*. Тогда последняя сессия будет короче. Остаток заполняйте нулями. То есть если в файле 24 записи и сессии длины 10, то 3 сессия будет состоять из 4 сайтов, и ей мы сопоставим вектор [*site1_id*, *site2_id*, *site3_id*, *site4_id*, 0, 0, 0, 0, 0, 0, *user_id*] 
- В итоге некоторые сессии могут повторяться – оставьте как есть, не удаляйте дубликаты. Если в двух сессиях все сайты одинаковы, но сессии принадлежат разным пользователям, то тоже оставляйте как есть, это естественная неопределенность в данных
- Не оставляйте в частотном словаре сайт 0 (уже в конце, когда функция возвращает этот словарь)
- 150 файлов из *capstone_websites_data/150users/* у меня обработались за 1.7 секунды, но многое, конечно, зависит от реализации функции и от используемого железа. И вообще, первая реализация скорее всего будет не самой эффективной, дальше можно заняться профилированием (особенно если планируете запускать этот код для 3000 пользователей). Также эффективная реализация этой функции поможет нам на следующей неделе.

In [46]:
def get_user_id(filename):
    '''Получает ID пользователя из названия файла. Возвращает целое число.'''
    
    extension = '.csv'
    len_number = 4
    end = filename.find(extension)
    start = end - len_number
    
    return int(filename[start:end])

def prepare_train_set(path_to_csv_files, session_length=10):
    '''
    Принимает на вход путь к каталогу с csv-файлами path_to_csv_files и параметр session_length. Каждый файл имеет в название user_id
    и содержит список посещенных сайтов, с указанием времени начала.
    Возвращает DataFrame, в котором строки соответствуют уникальным сессиям из session_length сайтов, session_length столбцов – индексам этих session_length сайтов
    и последний столбец – ID пользователя и частотный словарь сайтов вида {'site_string': [site_id, site_freq]}.
    '''
    site_freq = {} # создаем пустой словарь
    
    list_sessions = []

    for userfile in tqdm(glob(os.path.join(path_to_csv_files, '*.csv'))): # обходим все файлы в каталоге с расшарением .csv

        sites = pd.read_csv(userfile).site.values # читаем названия сайтов из файла
        for site in sites:
            if site not in site_freq: # проверяем наличие сайта в частотном словаре
                site_freq[site] = [len(site_freq) + 1, 0] # если такого сайта еще нет в словраре, то создаем такой элемент
            site_freq[site][1] +=1 # увеличиваем счетчик частоты повторений сайта 
        
        
        count_session = len(sites)//session_length if len(sites)%session_length == 0 else len(sites)//session_length+1

        for sindex in range(count_session):
            sessions = []
            for index in range(sindex * session_length, sindex * session_length + session_length):
                if index < len(sites):
                    sessions.append(site_freq[sites[index]][0])
                else:
                    sessions.append(0)
            sessions.append(get_user_id(userfile))
            list_sessions.append(sessions)
    
    
    headers = ['site' + str(index) for index in range(1, session_length+1)] + ['user_id'] # создаем список с заговолками для столбцов DataFrame
    train_data = pd.DataFrame(data=list_sessions, columns=headers) # создаем DataFrame
    
    return train_data, site_freq


In [47]:
prepare_train_set(PATH_TO_DATA + '/10users')

100%|██████████| 10/10 [00:00<00:00, 45.89it/s]


(       site1  site2  site3  site4  site5  site6  site7  site8  site9  site10  \
 0          1      1      2      3      4      3      5      5      6       7   
 1          7      8      9     10      9      9      9     11     12      13   
 2         14     15     15      7     16     17     18     19     20      21   
 3         21     22     23     24     22     25     19     22     14      15   
 4         15     22     22     22     14     15     22     22     25      22   
 5         15     14     15     22     22     22     15     14     15      22   
 6         22     25     19     22     25     15     14     22     22      22   
 7         22     14     15     15     22     22     14     15     15      22   
 8         22     22     19     15     14     14     15     14     15      14   
 9         14     14     15     14     25      3      4      3      5       5   
 10         5      7     24     21      9     10      7     26     27      28   
 11         5      5      5 

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [20]:
!cat $PATH_TO_DATA/3users/user0001.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,geo.mozilla.org
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,accounts.google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,apis.google.com
2013-11-15 11:41:35,plus.google.com
2013-11-15 12:40:35,vk.com
2013-11-15 12:40:37,google.com
2013-11-15 12:40:40,google.com
2013-11-15 12:41:35,google.com


In [21]:
!cat $PATH_TO_DATA/3users/user0002.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,football.kulichki.ru
2013-11-15 11:37:26,football.kulichki.ru
2013-11-15 11:40:32,oracle.com


In [22]:
!cat $PATH_TO_DATA/3users/user0003.csv

timestamp,site
2013-11-15 09:28:17,meduza.io
2013-11-15 09:33:04,google.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,google.com
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,yandex.ru
2013-11-15 11:41:35,meduza.io
2013-11-15 12:28:17,meduza.io
2013-11-15 12:33:04,google.com
2013-11-15 12:52:48,oracle.com


In [23]:
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

100%|██████████| 3/3 [00:00<00:00, 650.35it/s]


In [24]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,1,2,3,3,2,0,0,0,0,0,2
1,4,5,2,5,2,5,5,6,7,4,3
2,4,5,2,0,0,0,0,0,0,0,3
3,1,2,2,8,2,5,9,6,10,11,1
4,1,5,5,5,0,0,0,0,0,0,1


Частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой (первые элементы кортежей могут отличаться).

In [25]:
site_freq_3users

{'accounts.google.com': [9, 1],
 'apis.google.com': [10, 1],
 'football.kulichki.ru': [3, 2],
 'geo.mozilla.org': [8, 1],
 'google.com': [5, 9],
 'mail.google.com': [6, 2],
 'meduza.io': [4, 3],
 'oracle.com': [2, 8],
 'plus.google.com': [11, 1],
 'vk.com': [1, 3],
 'yandex.ru': [7, 1]}

Примените полученную функцию к данным по 10 пользователям.

**<font color='red'> Вопрос 1. </font> Сколько уникальных сессий из 10 сайтов в выборке с 10 пользователями?**

In [26]:
train_data_10users, site_freq_10users = prepare_train_set(PATH_TO_DATA + '/10users')

100%|██████████| 10/10 [00:00<00:00, 46.50it/s]


In [27]:
len(train_data_10users)

14061

**<font color='red'> Вопрос 2. </font> Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [28]:
len(site_freq_10users)

4913

Примените полученную функцию к данным по 150 пользователям.

**<font color='red'> Вопрос 3. </font> Сколько уникальных сессий из 10 сайтов в выборке с 150 пользователями?**

In [29]:
%%time
train_data_150users, site_freq_150users = prepare_train_set(PATH_TO_DATA + '/150users')

100%|██████████| 150/150 [00:02<00:00, 71.70it/s]


CPU times: user 2.14 s, sys: 206 ms, total: 2.35 s
Wall time: 2.34 s


In [30]:
len(train_data_150users)

137019

**<font color='red'> Вопрос 4. </font> Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [31]:
len(site_freq_150users)

27797

**<font color='red'> Вопрос 5. </font> Какой из этих сайтов НЕ входит в топ-10 самых популярных сайтов среди посещенных 150 пользователями?**
- www.google.fr
- www.youtube.com
- safebrowsing-cache.google.com
- www.linkedin.com

In [32]:
print(sorted(site_freq_150users.keys(), key=lambda x: site_freq_150users[x][1], reverse=True)[:10])

['www.google.fr', 'www.google.com', 'www.facebook.com', 'apis.google.com', 's.youtube.com', 'clients1.google.com', 'mail.google.com', 'plus.google.com', 'safebrowsing-cache.google.com', 'www.youtube.com']


<b>www.linkedin.com</b> - НЕ входит в топ-10 самых популярных сайтов

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [33]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. 

Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить. 

In [34]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [35]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,1,2,3,3,2,0,0,0,0,0,2
1,4,5,2,5,2,5,5,6,7,4,3
2,4,5,2,0,0,0,0,0,0,0,3
3,1,2,2,8,2,5,9,6,10,11,1
4,1,5,5,5,0,0,0,0,0,0,1


In [36]:
X_toy

array([[ 1,  2,  3,  3,  2,  0,  0,  0,  0,  0],
       [ 4,  5,  2,  5,  2,  5,  5,  6,  7,  4],
       [ 4,  5,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  2,  2,  8,  2,  5,  9,  6, 10, 11],
       [ 1,  5,  5,  5,  0,  0,  0,  0,  0,  0]])

In [37]:
def get_dense_matrix(matrix):
    '''
     Создает разряженную матрицу в которой строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов.
     На пересечении строки ii и столбца jj будет стоять число nijnij – сколько раз сайт jj встретился в сессии номер ii.
     Возвращает разряженную матрицу.
    '''
    site_ids = set(element for row in matrix for element in row if element != 0) # создаем список id сайтов
    
    i = 0
    data = []
    col = []
    rows = []

    for row in tqdm(matrix):
        
        unique, counts = np.unique(row, return_counts=True)
        
        temp_dict = dict(zip(unique, counts))
        
        for k in temp_dict:
            if k == 0:
                continue
            
            data.append(temp_dict[k])
            rows.append(i)
            col.append(k-1)
            
        i += 1
        
    X_sparse = csr_matrix((data, (rows, col)), shape=(matrix.shape[0], len(site_ids)))
    
    return X_sparse

In [38]:
X_sparse_toy = get_dense_matrix(X_toy)

100%|██████████| 5/5 [00:00<00:00, 1199.33it/s]


**Размерность разреженной матрицы должна получиться равной 11, поскольку в игрушечном примере 3 пользователя посетили 11 уникальных сайтов.**

In [39]:
X_sparse_toy.todense()

matrix([[1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 2, 4, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 3, 0, 0, 1, 1, 0, 1, 1, 1, 1],
        [1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0]])

In [40]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [41]:
X_10users

array([[  1,   1,   2, ...,   5,   6,   7],
       [  7,   8,   9, ...,  11,  12,  13],
       [ 14,  15,  15, ...,  19,  20,  21],
       ...,
       [ 19, 382,  25, ...,  18,  22,  19],
       [ 25, 382, 588, ...,  22,  25,  25],
       [382, 788, 180, ..., 180,  25, 745]])

In [42]:
X_sparse_10users = get_dense_matrix(X_10users)
X_sparse_150users = get_dense_matrix(X_150users)

100%|██████████| 137019/137019 [00:02<00:00, 46114.89it/s]


**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [43]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

**Чисто для подстраховки проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [44]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [45]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

## Пути улучшения
-  можно обработать исходные данные по 3000 пользователей; обучать на такой выборке модели лучше при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)). Хотя далее в курсе мы познакомимся с алгоритмами, способными обучаться на больших выборках при малых вычислительных потребностях;
- помимо явного создания разреженного формата можно еще составить выборки с помощью `CountVectorizer`, `TfidfVectorizer` и т.п. Поскольку данные по сути могут быть описаны как последовательности, то можно вычислять n-граммы сайтов. Работает все это или нет, мы будем проверять в [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle Inclass (желающие могут начать уже сейчас).

В следующей части мы еще немного поготовим данные и потестируем первые гипотезы, связанные с нашими наблюдениями. 